In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5d1046903e3dad02a4e662af69c7a7badd358381ad7682b214b7ed937b778128
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setMaster("local").setAppName("WikiMedia")

In [ ]:
sc = SparkContext(conf=conf)

In [ ]:
db= sc.textFile("/content/pagecounts-20160101-000000_parsed.out")
cnt= db.count()
print(cnt)

489156


point (1) max,min,avg with map-reduce

In [ ]:
lines = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = time.time()

page_size_rdd = lines.map(lambda line: int(line.split(" ")[-1]) if line.split(" ")[-1] != "null" else 0)
min_page_title = page_size_rdd.min()
max_page_title = page_size_rdd.max()
avg_page_title = page_size_rdd.mean()

# Create an RDD with tuples for each metric
writable_rdd = sc.parallelize([("min_page_title", min_page_title), ("max_page_title", max_page_title), ("avg_page_title", avg_page_title)])

# Save the RDD as a text file
writable_rdd.saveAsTextFile('./Q1-mapreduce')

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
print("min: ",min_page_title)
print("max: ",max_page_title)
print("average: ",avg_page_title)


Elapsed time:  29.1927169799805
min:  0
max:  141180155987
average:  132239.5695744666


point (1) max,min,avg with loop

In [ ]:
import numpy as np
import time
start_time = time.time()
# create an empty list to store page sizes
page_sizes = []

# iterate over each line in the RDD
for line in db.toLocalIterator():
    # extract the page size from the line
    if len(line.split(" ")) != 1:
      _page_size = line.split(" ")[-1]
      if _page_size == "null":
        page_sizes.append(int(0))
      else:
        page_sizes.append(int(_page_size))
    else:
      continue




# find the minimum, maximum, and average page sizes
min_page_title = float('inf')
max_page_title = float('-inf')
sum_page_title = 0
count = 0

for value in page_sizes:
    # Update minimum value
    if value < min_page_title:
        min_page_title = value

    # Update maximum value
    if value > max_page_title:
        max_page_title = value

    # Accumulate sum
    sum_page_title += value
    count += 1

avg_page_title = sum_page_title / count

writable_rdd = sc.parallelize([("min_page_title", min_page_title), ("max_page_title", max_page_title), ("avg_page_title", avg_page_title)])

# Save the RDD as a text file
writable_rdd.saveAsTextFile('./Q1--loop')

print("Minimum page title size:", min_page_title)
print("Maximum page title size:", max_page_title)
print("Average page title size:", avg_page_title)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Minimum page title size: 0
Maximum page title size: 141180155987
Average page title size: 132239.56957446598
Elapsed time: 34.78737950325012 seconds


point(2) the,en with map-reduce

In [ ]:
import time

start_time = time.time()
lines = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

if lines.isEmpty():
    print("No data found in the file.")
else:
    page_title_with_code_rdd = lines.map(lambda line: int(line.split(" ")[-1]) if line.split(" ")[-1] != "null" else 0)

    page_title_with_the_noten = lines.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
        .filter(lambda line: line[1][0:4] == 'The_' and line[0] != 'en')

    page_title_with_the = lines.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
        .filter(lambda line: line[1][0:4] == 'The_')

    count_the_noten = page_title_with_the_noten.count()
    count_the = page_title_with_the.count()
    # count_the_noten= count_the - count_the_en

    writable_rdd = sc.parallelize([ count_the_noten, count_the])

    writable_rdd.saveAsTextFile('./Q2-mapreduce')

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time)
print("started with the: ",count_the)
print("started with the and not english: ",count_the_noten)


Elapsed time: 21.21088194847107
started with the:  41901
started with the and not english:  9160


point(2) the,en with loop

In [ ]:
import time
from pyspark import SparkContext


sc = SparkContext.getOrCreate()

# Read the file into an RDD
lines = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = time.time()

# Initialize counters
count_the = 0
count_the_noten = 0

# Loop through the RDD and apply the filter and count operations
for line in lines.toLocalIterator():
    # Split the line into language code and page title
    parts = line.split(" ")
    if len(parts) >= 2:
        code, title = parts[0], parts[1]
        # Check if the page title starts with "The_" and the language code is not "en"
        if title.startswith("The_"):
            count_the += 1
            if code != 'en':
                count_the_noten += 1

writable_rdd = sc.parallelize([ count_the_noten, count_the])

writable_rdd.saveAsTextFile('./Q2-loop')

# Get the count of pages
print("start with The_: ", count_the)
print("started with the and not english: ", count_the_noten)

end_time = time.time()
elapsed_time = end_time - start_time

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")


start with The_:  41901
started with the and not english:  9160
Elapsed time: 34.10086393356323 seconds


point(3) unique with map-reduce

In [ ]:
import time
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

start_time = time.time()

# Read the file into an RDD
lines = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

# Extract page titles
page_title_rdd = lines.map(lambda line: line.split(" ")[1] if len(line.split(" ")) >= 2 else "")

# Split titles into terms by '_'
splited_title = page_title_rdd.flatMap(lambda title: title.split('_'))

# Normalize terms: lowercase and remove non-alphanumeric characters
normalized_splited_title = splited_title.map(lambda term: term.lower())
normalized_splited_title = normalized_splited_title.map(lambda term: ''.join(ch for ch in term if ch.isalnum()))

# Count occurrences of each term
from operator import add
term_count = normalized_splited_title.map(lambda term: (term, 1)).reduceByKey(add)

# Filter to get distinct terms (terms that appear exactly once)
distinct_terms = term_count.filter(lambda term: term[1] == 1)

# Count distinct terms
cnt = distinct_terms.count()

# Create an RDD with the count and save it as a text file
writable_rdd = sc.parallelize([cnt])
writable_rdd.saveAsTextFile('./Q3-mapreduce')

end_time = time.time()
elapsed_time = end_time - start_time

# Print the elapsed time and count of unique terms
print("Elapsed time:", elapsed_time)
print("Number of unique terms:", cnt)


Elapsed time: 57.93020009994507
Number of unique terms: 1292290


point(3) unique with loop


In [ ]:
import time
start_time = time.time()
title_list = []
for line in page_title_rdd.toLocalIterator():
  if len(line.split(" ")) >= 2:
    title = line.split(" ")[1]
    title_list.append(title)

# split titles into words

splited_title = []
for line in page_title_rdd.toLocalIterator():
    words = line.split('_')
    splited_title.extend(words)

normalized_terms = []
for term in splited_title:
    term_normalized = term.lower()
    term_normalized = ''.join(ch for ch in term_normalized if ch.isalnum())
    normalized_terms.append(term_normalized)


# normalize terms

# count distinct terms
term_count = {}
for term in normalized_splited_title.toLocalIterator():
    if term not in term_count:
        term_count[term] = 1
    else:
        term_count[term] += 1

distinct_terms = sc.parallelize([(term, count) for term, count in term_count.items() if count == 1])

# get count of distinct terms
count = 0
for term in distinct_terms.toLocalIterator():
    count += 1
print(count)

writable_rdd = sc.parallelize([count])
writable_rdd.saveAsTextFile('./Q3-loop')
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

1292290
Elapsed time: 84.58370614051819 seconds


point(4) extract with map-reduce

In [ ]:
import time
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext.getOrCreate()

start_time = time.time()

# Extract page titles
page_title_rdd = db.map(lambda line: line.split(" ")[1] if len(line.split(" ")) >= 2 else line.split(" ")[0])

# Count the frequency of each title
freq_title = page_title_rdd.map(lambda term: (term, 1)).reduceByKey(lambda a,b:a +b)

# Sort titles by frequency in descending order
most_freq_title = freq_title.sortBy(lambda pair: pair[1], ascending=False)

# Print the results using foreach
most_freq_title.foreach(lambda x: print(f"({x[0]}, {x[1]})"))

# Save the most frequent title and its frequency as a text file
most_freq_title.map(lambda x: f"{x[0]}, {x[1]}").saveAsTextFile('./Q4-mapreduce')

# Get the most frequent title
most_freq = most_freq_title.first()

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Elapsed time: 14.398543119430542 seconds


point(4) extract with loop

In [ ]:
import time
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

start_time = time.time()

# Count the frequency of each title
title_counts = {}
for line in db.toLocalIterator():
    fields = line.split(" ")
    if len(fields) >= 2:
        title = fields[1]
    else:
        title = fields[0]

    if title in title_counts:
        title_counts[title] += 1
    else:
        title_counts[title] = 1

# Sort the title counts in descending order
sorted_counts = sorted(title_counts.items(), key=lambda item: item[1], reverse=True)

# Print the results
for title, count in sorted_counts:
    print(f"({title}, {count})")

# Save the results to a file
with open('./Q4-loop', 'w') as f:
    for title, count in sorted_counts:
        f.write(f"({title}, {count})\n")

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Streaming output truncated to the last 5000 lines.
(File:Stenopterygius_quadriscissus_6767.JPG, 1)
(File:Stensioella_heintzi.jpg, 1)
(File:Stensjon.jpg, 1)
(File:Step3-specify.JPG, 1)
(File:Step_Up_-_geograph.org.uk_-_691124.jpg, 1)
(File:Stepan_Blois.jpg, 1)
(File:Stepanaket_At_Night-1.JPG, 1)
(File:Stephan_Agricola.jpg, 1)
(File:Stephan_Huber,_Gran_Paradiso,_Neue_Messe_Muenchen-1.jpg, 1)
(File:Stephan_Pilarick.jpg, 1)
(File:Stephan_Sinding-Verwitwet.jpg, 1)
(File:Stephan_Urbach_ACTAvist.jpg, 1)
(File:Stephan_toscani.jpg, 1)
(File:StephaneGrappelli25.JPG, 1)
(File:Stephanie_Radziwil_(1809-1832),_by_Russian_School_of_the_19th_century.jpg, 1)
(File:Stephansdom_O5.jpg, 1)
(File:Stephanus_Schoeman.jpg, 1)
(File:StephenPearson01.JPG, 1)
(File:Stephen_Amell.jpg, 1)
(File:Stephen_Amell_WonderCon_2013_(Straighten_Crop).jpg, 1)
(File:Stephen_Austin_by_Elisabet_Ney,1905.jpg, 1)
(File:Stephen_C._Foster_State_Park_flagpole.JPG, 1)
(File:Stephen_Curry_shooting.jpg, 1)
(File:Stephen_Fleming_slip.jp

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
(.44mag, 1)
(.45-70, 1)
(.45-70_Government, 1)
(.450/400_Nitro_Express, 1)
(.450_Bushmaster, 1)
(.450_Dakota, 1)
(.450_Marlin, 1)
(.450_No_2_Nitro_Express, 1)
(.450_Rigby, 1)
(.455_Webley, 1)
(.458_Express, 1)
(.458_Lott, 1)
(.458_SOCOM, 1)
(.458_Winchester_Magnum, 1)
(.45_(film), 1)
(.45_Auto_Rim, 1)
(.45_Magnum, 1)
(.45_Mars_Long, 1)
(.45_Schofield, 1)
(.45_Super, 1)
(.45_caliber_ammunition, 1)
(.460_Rowland, 1)
(.470_Nitro_Express, 1)
(.475, 1)
(.475_Linebaugh, 1)
(.475_No._2_Nitro_Express, 1)
(.475_Wildey_Magnum, 1)
(.480_Ruger, 1)
(.50-110_Winchester, 1)
(.50-70, 1)
(.50-70_Government, 1)
(.50-90_Sharps, 1)
(.50-95_Express, 1)
(.500_A-Square, 1)
(.500_Black_Powder_Express, 1)
(.500_Jeffery, 1)
(.500_Linebaugh, 1)
(.500_Maximum, 1)
(.500_Nitro_Express, 1)
(.500_No._2_Express, 1)
(.500_SW, 1)
(.500_Wyoming_Express, 1)
(.505_Gibbs, 1)
(.50BMG, 1)
(.50_AE, 1)
(.50_Alaskan, 1)
(.50_BMG, 1)
(.50_Beowulf, 1)
(.50_Caliber_BMG_Regulation_A

point(5) combine map-reduce

In [ ]:
import time
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

db = sc.textFile("/content/pagecounts-20160101-000000_parsed.out")

start_time = time.time()

# Function to parse each line and extract necessary fields
def parse_line(line):
    fields = line.split(" ")
    title = fields[1] if len(fields) >= 2 else fields[0]
    hinst = fields[2:-1] if len(fields) > 3 else []
    size = fields[-1] if len(fields) >= 2 else '0'
    return (title, (hinst, [size]))

# Parse, combine, and reduce the data
combined_data = db.map(parse_line) \
    .reduceByKey(lambda acc, value: (acc[0] + value[0], acc[1] + value[1])) \
    .mapValues(lambda x: {"hinst": ",".join(x[0]), "size": x[1]})

# Format the output for saving
formatted_output = combined_data.map(lambda x: f"(Title: {x[0]}\nPage data:\n    Hinst: {x[1]['hinst']}\n    Size: {','.join(x[1]['size'])})\n\n")

# Save the output to a file
formatted_output.saveAsTextFile("Q5-mapreduce")

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Elapsed time: 5.699732303619385 seconds


point(5) combine with loop


In [ ]:


# Parse the data and store in a list
parsed_data = []
for line in db.toLocalIterator():
    fields = line.split(" ")
    if len(fields) >= 3:
        title = fields[1]
        hinst = fields[2:-1]  # Extract hinst values
        size = fields[-1]  # Extract size value
        parsed_data.append((title, (hinst, [size])))

# Combine data for pages with the same title
combined_data = {}
for title, data in parsed_data:
    hinst, size = data
    if title not in combined_data:
        combined_data[title] = ([], [])
    combined_data[title][0].extend(hinst)
    combined_data[title][1].extend(size)

# Format the results for display
result = {}
for title, data in combined_data.items():
    hinst = ", ".join(data[0])
    size = ", ".join(data[1])
    result[title] = {"hinst": hinst, "size": size}

with open('./Q5-loop', 'w') as f:
    for title, data in result.items():
        f.write(f"({title}, {data})\n")

# Print the results
for title, data in result.items():
    hinst = data["hinst"]
    size = data["size"]
    print(f"(Title: {title}\nPage data:\n    Hinst: {hinst}\n    Size: {size})\n\n")

end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Streaming output truncated to the last 5000 lines.
    Size: 9173)


(Title: Primula_suffrutescens
Page data:
    Hinst: 1
    Size: 8584)


(Title: Prinanses
Page data:
    Hinst: 1
    Size: 19813)


(Title: Prionus_palparis
Page data:
    Hinst: 1
    Size: 10014)


(Title: Prionyx
Page data:
    Hinst: 1
    Size: 9629)


(Title: Prionyx_funebris
Page data:
    Hinst: 1
    Size: 8101)


(Title: Priotyrannus_closteroides
Page data:
    Hinst: 1
    Size: 8026)


(Title: Pristimantis_cruentus.html
Page data:
    Hinst: 1
    Size: 4945)


(Title: Pristimantis_devillei
Page data:
    Hinst: 1
    Size: 9008)


(Title: Pristimantis_jester
Page data:
    Hinst: 1
    Size: 8825)


(Title: Pristina_aequiseta
Page data:
    Hinst: 1
    Size: 7895)


(Title: Pristurus_adrarensis
Page data:
    Hinst: 1
    Size: 8032)


(Title: Procambarus_verrucosus
Page data:
    Hinst: 1
    Size: 8801)


(Title: Prochoristis
Page data:
    Hinst: 1
    Size: 8467)


(Title: Prodiamesa
Page data:
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.


(Title: File:Chateau-decouen.jpg
Page data:
    Hinst: 2
    Size: 53488)


(Title: File:Chateau-facace-sud.JPG
Page data:
    Hinst: 1
    Size: 9418)


(Title: File:Chateau-lagrezette_2.jpg
Page data:
    Hinst: 1
    Size: 9125)


(Title: File:Chateau.Milan.png
Page data:
    Hinst: 2
    Size: 0)


(Title: File:ChateauMarmont_01.jpg
Page data:
    Hinst: 1
    Size: 9903)


(Title: File:ChateauThizy.JPG
Page data:
    Hinst: 1
    Size: 9580)


(Title: File:Chateau_Arlay_02.jpg
Page data:
    Hinst: 1
    Size: 8631)


(Title: File:Chateau_Beaulard.jpg
Page data:
    Hinst: 1
    Size: 8633)


(Title: File:Chateau_Busigny_avant_2007.jpg
Page data:
    Hinst: 1
    Size: 8421)


(Title: File:Chateau_Bussy_Rabutin_26.JPG
Page data:
    Hinst: 1
    Size: 9330)


(Title: File:Chateau_Chalais-erd5.jpg
Page data:
    Hinst: 1
    Size: 10580)


(Title: File:Chateau_Flers_4.jpg
Page data:
    Hinst: 1
    Size: 10035)


(Title: File:Ch

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.

(Title: File:Netball_pictogram.svg
Page data:
    Hinst: 1
    Size: 52892)


(Title: File:Netent_logo.jpg
Page data:
    Hinst: 1
    Size: 6129)


(Title: File:Netflix_2014_logo.svg
Page data:
    Hinst: 2
    Size: 0)


(Title: File:Netflix_logo.svg
Page data:
    Hinst: 1
    Size: 9366)


(Title: File:Netgiraffe_(4528768166)_(3).jpg
Page data:
    Hinst: 1
    Size: 10140)


(Title: File:Nethack_falcons_eye_class_selection.png
Page data:
    Hinst: 1
    Size: 8890)


(Title: File:Nethack_falcons_eye_race_selection.png
Page data:
    Hinst: 1
    Size: 8846)


(Title: File:Nether_Booth_Farm_-_geograph.org.uk_-_226196.jpg
Page data:
    Hinst: 1
    Size: 11112)


(Title: File:Netherlands,_Leidschendam-Voorburg,_Rietpolder_(1).png
Page data:
    Hinst: 1
    Size: 9752)


(Title: File:Netherlands._Schiedam_SA_Band,_1898.jpg
Page data:
    Hinst: 1
    Size: 8503)


(Title: File:NetherlandsGlobe.gif
Page data:
    Hinst: 1
    Size